#  Setup: Display options

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


# Setup: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [8]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [9]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 2N2rvg7aV5xgqMbbnyJvpGWjkG9mZe32g9MxV6pVyM3S, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:
# ##################################################################

In [ ]:

#  Making a GS Bucket Public,
#
#  From,
#     https://cloud.google.com/storage/docs/access-control/making-data-public#command-line

gsutil iam ch allUsers:objectViewer gs://farrell-bucket


#  In browser,
#
#     https://console.cloud.google.com/storage/browser/farrell-bucket


In [ ]:

#  Read CSV from GS

#  import os
   #
import dask.dataframe as dd
import numpy as np
   #
#  from google.cloud import storage
#  import gcsfs


#  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/03_MyKeys.json"


l_bucket    = "katana-demo-datasets"
l_fullpath  = "gs://" + l_bucket + "/csv-datasets/ldbc/sf-0.003/csv/bi/composite-projected-fk/initial_snapshot/static/Place/part-00000-c16729dd-16d5-456b-84ca-d0f19fa783d6-c000.csv"


l_dataframe = dd.read_csv(
   l_fullpath,
   delimiter='|',
   dtype={
      'id':   int,
      "name": np.dtype('O'),
      "url":  np.dtype('O'),
      "type": np.dtype('O'),
      })


l_cntr = 0
   #
for l_each in l_dataframe.itertuples():
   l_cntr+=1
      #
   if (l_cntr < 10):
      print("Id: %-4d    Name: %-28s   Url: %-60s   Type: %s" % (l_each.id, l_each.name, l_each.url, l_each.type))
    
print("")
    

In [ ]:

#  Output the above to GS Parquet
#
#     See,
#        https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-list-gcloud
#
#        gcloud iam service-accounts keys list --iam-account=farrell@katana-internal39.iam.gserviceaccount.com


import os
   #
import dask.dataframe as dd
import numpy as np
   #
from google.cloud import storage
import gcsfs


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/03_MyKeys.json"




l_outputfolder = "gs://" + "farrell-bucket" + "/test"


l_dataframe.to_parquet(l_outputfolder,
   write_index         = False,
   write_metadata_file = False,
      #
   engine              = "pyarrow",
   version             = "2.6"
   )

print("--")


In [ ]:

#  Read the above

df_data = dd.read_parquet("https://storage.googleapis.com/farrell-bucket/test/part.0.parquet")


#  df_data.head(5)

l_cntr = 0
   #
for l_each in df_data.itertuples():
   l_cntr += 1
      #
   if (l_cntr < 5):
      print("Id: %-4d    Name: %-28s   Url: %-60s   Type: %s" % (l_each.id, l_each.name, l_each.url, l_each.type))

print("--")    


In [ ]:

#  Import GS Parquet data into graph

#  Currently this fails with sa missing column reference
#
from katana.remote import import_data

l_node ="gs://farrell-bucket/test/50_NodeHeader.txt"
l_data ="gs://farrell-bucket"

import_data.parquet(
   my_graph,
      #
   input_node_path   = l_node,
   #  input_edge_path   = l_edge,
   input_dir         = l_data,
      #
   schema_delimiter="|",
   ids_are_integers=True
   )

print("--")
